In [1]:
import os
import math

import numpy as np
import random

from tqdm import tqdm
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt

import torch

from sklearn.metrics import confusion_matrix, average_precision_score, roc_curve, roc_auc_score, accuracy_score,f1_score, normalized_mutual_info_score

from model.models import LBG
from model.temporalNeighbor import temporalNeighbor
from logger import logger
from collections import Counter
from sklearn.preprocessing import StandardScaler
#聚类指标：轮廓系数
from sklearn.metrics import silhouette_score

### Utility function and class
class EarlyStopMonitor(object):
    def __init__(self, max_round=5, higher_better=True, tolerance=1e-3):
        self.max_round = max_round
        self.num_round = 0

        self.epoch_count = 0
        self.best_epoch = 0

        self.last_best = None
        self.higher_better = higher_better
        self.tolerance = tolerance

    def early_stop_check(self, curr_val):

        if not self.higher_better:
            curr_val *= -1
        if self.last_best is None:
            self.last_best = curr_val
        elif (curr_val - self.last_best) / np.abs(self.last_best) < -self.tolerance:
            self.last_best = curr_val
            self.num_round = 0
            self.best_epoch = self.epoch_count
        else:
            self.num_round += 1
        self.epoch_count += 1
        print('last_best,best_epoch',self.last_best,self.best_epoch)
        return self.num_round >= self.max_round

class RandEdgeSampler(object):
    def __init__(self, src_list, dst_list):
        self.src_list = np.unique(src_list)
        self.dst_list = np.unique(dst_list)

    def sample(self, size):
        src_index = np.random.randint(0, len(self.src_list), size)
        dst_index = np.random.randint(0, len(self.dst_list), size)
        return self.src_list[src_index], self.dst_list[dst_index]

SEED = 1
ADGAT_EPOCHS = 150
BATCH_SIZE = 2000
NUM_LAYER = 1
UNIFORM = True
GPU = 1
NUM_NEIGHBORS = 24
NUM_HEADS = 16
DROP_OUT = 0.1
input_feat = 128
output_feat = 128
hidden = 128
USE_TIME = 'time'
AGG_METHOD = 'attn'
ATTN_MODE = 'prod'
objective = 'soft-boundary' #'hard'
SEQ_LEN = NUM_NEIGHBORS
LEARNING_RATE = 1e-3
DATA = 'LANL'
c = None
R=0
RR = 20#threshold
nu = 1e-5 

logger.info('\n---------------------------------------new start training--------------------------------------\n')
logger.info('parameter,ADGAT_EPOCHS:{},BATCH_SIZE:{},NUM_LAYER:{}'.format(ADGAT_EPOCHS,BATCH_SIZE,NUM_LAYER))
logger.info('parameter,NUM_NEIGHBORS:{},NUM_HEADS:{},DROP_OUT:{},input_feat:{}'.format(NUM_NEIGHBORS,NUM_HEADS,DROP_OUT,input_feat))

ALLEDGEPATH= './data/LANL/exampleAIG_LANL.txt'#41590401,4603678
ADGATMODEL_SAVE_PATH = f'./save_models/{AGG_METHOD}-{ATTN_MODE}-{DATA}.pth'
ADGATget_checkpoint_path = lambda epoch: f'./save_checkpoints/{AGG_METHOD}-{ATTN_MODE}-{DATA}-{epoch}.pth'
red_team_name = './data/LANL/nredteam.txt'
data_name =  './data/LANL/TCG_LANL.txt'
precess_file_path =  './data/LANL/iniACG_LANL.txt'
logger.info('current file:{}'.format(ALLEDGEPATH))

def setup_seed(seed):
    if torch.cuda.is_available():
        logger.info("gpu cuda is available!")
        torch.cuda.manual_seed_all(seed)
    else:
        logger.info("cuda is not available! cpu is available!")
        torch.manual_seed(seed)
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    os.environ['PYTHONHASHSEED'] = str(seed)


setup_seed(SEED)
red_team_file = set(open(red_team_name, 'r').readlines())


def model_roc_curve(test_node,FPR, TPR):
    fig_roc = plt.figure()
    plt.plot([0, 1], [0, 1], 'k--')
    A, = plt.step(FPR, TPR, color="#ff5722", linestyle='-.', alpha=1, where='post', label="roc")
    font1 = {'family': 'Times New Roman',
             'weight': 'normal',
             'size': 16,
             }
    font2 = {'family': 'Times New Roman',
             'weight': 'normal',
             'size': 13,
             }
    plt.tick_params(labelsize=12)
    plt.xlabel('False positive rate', font1)
    plt.ylabel('True positive rate', font1)
    plt.ylim([0.0, 1.05])
    plt.xlim([0.0, 1.0])

    legend = plt.legend(handles=[A], prop=font2)

    plt.title('ROC curve', fontsize=14)

    fig_roc.savefig(test_node+".png", dpi=500, bbox_inches='tight')


def load_data(file):
    edge_train = []
    label_src = []
    label_dst = []
    node_id = set()
    train_src_l = []
    train_dst_l = []
    train_ts_l = []
    train_label_l = []
    i = 0
    with open(file, 'r', encoding='utf-8') as fi:
        while True:
            line = fi.readline()
            if not line:
                break
            l = line.strip().split(',')
            if (int(l[0]) > 150884):#72-83
                break
            str_line = str(l[0]) + ',' + str(l[1]) + ',' + str(l[2]) + ',' + str(l[3]) + ',' + str(l[6]) + '\n'
            if str_line in red_team_file:
                train_label_l.append(1)
            else:
                train_label_l.append(0)
            C1 = int(l[3].strip('C'))
            C2 = int(l[6].strip('C'))
            node_id.add(C1)
            node_id.add(C2)
            i = i + 1

            edge_train.append([C1, C2, int(l[0])])
            train_src_l.append(C1)
            label_src.append(0)
            train_dst_l.append(C2)
            label_dst.append(0)
            train_ts_l.append(int(l[0]))

    return node_id, train_src_l, train_dst_l, train_ts_l, train_label_l, label_src, label_dst


node_id, train_src_l, train_dst_l, train_ts_l, train_label_l, label_src, label_dst = load_data(ALLEDGEPATH)

feat_edge = np.zeros((len(train_src_l), input_feat))
max_idx = max(30000, max(node_id))  #max(node_id)27193

feat_node = np.random.random((max_idx, input_feat))
feat_nodes = feat_node
Mm = StandardScaler()
feat_node_nor = Mm.fit_transform(feat_nodes)


assert (len(train_src_l) == len(train_dst_l))
assert (len(train_dst_l) == len(train_ts_l))
#assert(len(train_ts_l)==len(feat_edge))
train_e_idx_l = list(np.arange(len(train_src_l)))
logger.info("Finish Loading Data")



2024-09-02 04:28:55,701 140564686231360 2959834583.py[line:87] - INFO: 
---------------------------------------new start training--------------------------------------

2024-09-02 04:28:55,703 140564686231360 2959834583.py[line:88] - INFO: parameter,ADGAT_EPOCHS:150,BATCH_SIZE:2000,NUM_LAYER:1
2024-09-02 04:28:55,704 140564686231360 2959834583.py[line:89] - INFO: parameter,NUM_NEIGHBORS:24,NUM_HEADS:16,DROP_OUT:0.1,input_feat:128
2024-09-02 04:28:55,705 140564686231360 2959834583.py[line:97] - INFO: current file:./data/LANL/exampleAIG_LANL.txt
2024-09-02 04:28:55,727 140564686231360 2959834583.py[line:101] - INFO: gpu cuda is available!
2024-09-02 04:28:56,261 140564686231360 2959834583.py[line:196] - INFO: Finish Loading Data


In [2]:
adj_list = [[] for _ in range(max_idx + 1)]
train_src_l = np.array(train_src_l)
train_dst_l = np.array(train_dst_l)
train_ts_l = np.array(train_ts_l)
train_label_l = np.array(train_label_l)
label_src = np.array(label_src)
label_dst = np.array(label_dst)
# NUM_HEADS = 4
### Initialize the data structure for graph and edge sampling
for src, dst, eidx, ts in zip(train_src_l, train_dst_l, train_e_idx_l, train_ts_l):
    adj_list[src].append((dst, eidx, ts))
train_ngh_finder = temporalNeighbor(adj_list, uniform=UNIFORM)
### Model initialize
device = torch.device('cuda:{}'.format(GPU))
lbg = LBG(train_ngh_finder, feat_node_nor,num_layers=NUM_LAYER, use_time=USE_TIME,
            agg_method=AGG_METHOD, attn_mode=ATTN_MODE, seq_len=SEQ_LEN, n_head=NUM_HEADS,
            drop_out=DROP_OUT, node_dim=hidden, time_dim=hidden)
# optimizer = torch.optim.SGD(lbg.parameters(), lr=LEARNING_RATE)
optimizer = torch.optim.SGD(lbg.parameters(), lr=LEARNING_RATE, weight_decay=1e-4)
lbg = lbg.to(device)

num_instance = len(train_src_l)
num_batch = math.ceil(num_instance / BATCH_SIZE)
logger.info('num of training instances: {}'.format(num_instance))
logger.info('num of batches per epoch: {}'.format(num_batch))
logger.info('num of red team: {}'.format(sum(train_label_l)))

train_rand_sampler = RandEdgeSampler(train_src_l, train_dst_l)

idc = np.arange(num_instance)


def Deep_SVDD(output, R, c, nu):
    dist = torch.sum((output-c) ** 2, dim=1)
    # if objective == 'soft-boundary':
    #     scores = dist - R ** 2
    #     loss = R ** 2 + (1 / nu) * torch.mean(torch.max(torch.zeros_like(scores), scores))
    # else:
    scores = dist
    loss = torch.mean(dist)
    return scores,loss

def update_center(src_l, dst_l,ts_l,lbg,num_instance,num_batch):
    c = torch.zeros(2 * (output_feat)).to(device)
    eps = 0.1
    lbg = lbg.eval()
    n_samples = 0
    idx = np.arange(num_instance)
    with torch.no_grad():
        for k in tqdm(range(num_batch)):
            s_idx = k * BATCH_SIZE
            e_idx = min(num_instance - 1, s_idx + BATCH_SIZE)
            if (s_idx == e_idx):
                continue
            batch_idx = idx[s_idx:e_idx]
            src_l_cut, dst_l_cut,ts_l_cut = src_l[batch_idx], dst_l[batch_idx],ts_l[batch_idx]
            enc = lbg(src_l_cut, dst_l_cut, ts_l_cut, NUM_NEIGHBORS)
            n_samples += enc.shape[0]
            c += torch.sum(enc, dim=0)
        c /= n_samples
        c[(abs(c) < eps) & (c < 0)] = -eps
        c[(abs(c) < eps) & (c > 0)] = eps
    return c
center = update_center( train_src_l, train_dst_l,train_ts_l,lbg,num_instance,num_batch).to(device)
ADGATearly_stopper = EarlyStopMonitor()




2024-09-02 04:28:57,295 140564686231360 3827178830.py[line:24] - INFO: num of training instances: 126284
2024-09-02 04:28:57,296 140564686231360 3827178830.py[line:25] - INFO: num of batches per epoch: 64
2024-09-02 04:28:57,304 140564686231360 3827178830.py[line:26] - INFO: num of red team: 0


100%|██████████| 64/64 [00:03<00:00, 21.03it/s]


In [3]:
def get_radius(dist: torch.Tensor, nu: float):
    """Optimally solve for radius R via the (1-nu)-quantile of distances."""
    return np.quantile(np.sqrt(dist.clone().data.cpu().numpy()), 1 - nu)

j=1
for epoch in (range(ADGAT_EPOCHS)):
    # Training
    # training use only training graph
    lbg.ngh_finder = train_ngh_finder
    acc, ap, f1, auc, m_loss, fpr, tpr, nmi,temR = [], [], [], [], [], [], [], [],[]
    pred_ll, true_ll, link_embed = [], [], []
    logger.info('start {} epoch'.format(epoch))
    idx = np.arange(num_instance)
    np.random.shuffle(idx)
    for k in tqdm(range(num_batch)):
        s_idx = k * BATCH_SIZE
        e_idx = min(num_instance - 1, s_idx + BATCH_SIZE)
        if (s_idx == e_idx):
            continue
        batch_idx = idx[s_idx:e_idx]
        src_l_cut, dst_l_cut, ts_l_cut= train_src_l[batch_idx], train_dst_l[batch_idx],train_ts_l[batch_idx]
        label_l_cut = train_label_l[batch_idx]
        size = len(src_l_cut)

        with torch.no_grad():
            pos_label = torch.from_numpy(label_l_cut).float().to(device)
        optimizer.zero_grad()

        lbg = lbg.train()
        link = lbg(src_l_cut, dst_l_cut, ts_l_cut, NUM_NEIGHBORS)
        score, loss = Deep_SVDD(link, R, center, nu)
        loss.backward()
        optimizer.step()
        link_embed.extend(link.cpu())
        m_loss.append(loss.item())
        with torch.no_grad():
            lbg = lbg.eval()
            pred_score = score.cpu().detach().numpy()
            true_label = pos_label.cpu().numpy()
            pred_ll.extend(list(pred_score))
            true_ll.extend(list(true_label))
    pred_ll = np.array(pred_ll)
    true_ll = np.array(true_ll)
    link_embed = torch.stack(link_embed)
    RR = get_radius(torch.sum((link_embed - center.cpu()) ** 2, dim=1), nu)
    logger.info('Epoch mean loss: {}'.format(np.mean(m_loss)))
    if ADGATearly_stopper.early_stop_check(np.mean(m_loss)):
        logger.info('Loss is no decline over {} epochs , stop training'.format(ADGATearly_stopper.max_round))
        logger.info(f'Loading the best model at epoch {ADGATearly_stopper.best_epoch}')
        best_model_path = ADGATget_checkpoint_path(ADGATearly_stopper.best_epoch)
        lbg.load_state_dict(torch.load(best_model_path))
        logger.info(f'Loaded the best model at epoch {ADGATearly_stopper.best_epoch} for inference')
        break
    else:
        torch.save(lbg.state_dict(), ADGATget_checkpoint_path(epoch))
logger.info('Traing endding')




2024-09-02 04:29:00,387 140564686231360 584914274.py[line:12] - INFO: start 0 epoch


100%|██████████| 64/64 [00:03<00:00, 16.99it/s]


2024-09-02 04:29:04,317 140564686231360 584914274.py[line:46] - INFO: Epoch mean loss: 68.46895635128021
last_best,best_epoch 68.46895635128021 0
2024-09-02 04:29:04,377 140564686231360 584914274.py[line:12] - INFO: start 1 epoch


100%|██████████| 64/64 [00:03<00:00, 16.03it/s]


2024-09-02 04:29:08,536 140564686231360 584914274.py[line:46] - INFO: Epoch mean loss: 50.78185302019119
last_best,best_epoch 50.78185302019119 1
2024-09-02 04:29:08,585 140564686231360 584914274.py[line:12] - INFO: start 2 epoch


...

2024-09-02 04:38:41,185 140564686231360 584914274.py[line:46] - INFO: Epoch mean loss: 3.165511406958103
last_best,best_epoch 3.165511406958103 147
2024-09-02 04:38:41,233 140564686231360 584914274.py[line:12] - INFO: start 148 epoch


100%|██████████| 64/64 [00:03<00:00, 16.03it/s]


2024-09-02 04:38:45,423 140564686231360 584914274.py[line:46] - INFO: Epoch mean loss: 3.1542757526040077
last_best,best_epoch 3.1542757526040077 148
2024-09-02 04:38:45,472 140564686231360 584914274.py[line:12] - INFO: start 149 epoch


100%|██████████| 64/64 [00:03<00:00, 16.28it/s]


2024-09-02 04:38:49,589 140564686231360 584914274.py[line:46] - INFO: Epoch mean loss: 3.1401260048151016
last_best,best_epoch 3.1401260048151016 149
2024-09-02 04:38:49,629 140564686231360 584914274.py[line:56] - INFO: Traing endding


In [4]:
def test(test_node,lbg,R,num_instance,src_l,dst_l,ts_l,label_l,center):
    logger.info("{}".format(test_node))
    acc, ap, f1, auc, m_loss, fpr, tpr, nmi = [], [], [], [], [], [], [], []
    link_embed= []
    pred_ll, true_ll = [], []
    with torch.no_grad():
        lbg = lbg.eval()
        BATCH_SIZE = 2000
        num_batch = math.ceil(num_instance / BATCH_SIZE)
        for k in tqdm(range(num_batch)):
            s_idx = k * BATCH_SIZE
            e_idx = min(num_instance - 1, s_idx + BATCH_SIZE)
            if (s_idx == e_idx):
                continue
            batch_idx = idc[s_idx:e_idx]
            src_l_cut, dst_l_cut,ts_l_cut = src_l[batch_idx], dst_l[batch_idx],ts_l[batch_idx]

            label_l_cut = label_l[batch_idx]

            pos_label = torch.from_numpy(label_l_cut).float().to(device)
            link = lbg(src_l_cut, dst_l_cut, ts_l_cut, NUM_NEIGHBORS)
            score,_= Deep_SVDD(link, R, center, nu)
            pred_score = score.cpu().detach().numpy()
            true_label = pos_label.cpu().numpy()
            pred_ll.extend(list(pred_score))
            true_ll.extend(list(true_label))
            link_embed.extend(link.cpu())
            m_loss.append(_.item())

        pred_ll = np.array(pred_ll)
        true_ll = np.array(true_ll)
        link_embed = torch.stack(link_embed)
        auc = roc_auc_score(true_ll, pred_ll)
        FPR_curve, TPR_curve, threshold1s = roc_curve(true_ll, pred_ll)
        logger.info("FPR_curve,: {},TPR_curve,:{}".format(FPR_curve, TPR_curve))
        tn, fp, fn, tp = confusion_matrix(true_ll, pred_ll>RR, labels=[False, True]).ravel()
        logger.info('Epoch mean loss: {}'.format(np.mean(m_loss)))
        logger.info("Confusion Matrix:{}".format(confusion_matrix(true_ll, pred_ll>RR, labels=[False, True])))
        FPR = fp / (fp + tn)
        TPR = tp / (tp + fn)
        logger.info("FPR: {}".format(FPR))
        logger.info("TPR: {}".format(TPR))
        print(FPR,TPR,auc)
        logger.info('test auc: {}'.format(auc))
        pred_label = pred_ll>RR
        redlink_idx = np.where(pred_label==1)[0]
        red_src_detect, red_dst_detect,red_tsl_detect = src_l[redlink_idx], dst_l[redlink_idx],ts_l[redlink_idx]
    return red_src_detect, red_dst_detect,red_tsl_detect,link_embed,true_ll

In [5]:
def load_data(file):
    edge_train = []
    label_src = []
    label_dst = []
    node_id = set()
    train_src_l = []
    train_dst_l = []
    train_ts_l = []
    train_label_l = []
    i = 0
    with open(file, 'r', encoding='utf-8') as fi:
        while True:
            line = fi.readline()
            if not line:
                break
            l = line.strip().split(',')
            if (int(l[0]) < 150885 ):#72-83
                 continue
            str_line = str(l[0]) + ',' + str(l[1]) + ',' + str(l[2]) + ',' + str(l[3]) + ',' + str(l[6]) + '\n'
            if str_line in red_team_file:
                train_label_l.append(1)
            else:
                train_label_l.append(0)
            C1 = int(l[3].strip('C'))
            C2 = int(l[6].strip('C'))
            node_id.add(C1)
            node_id.add(C2)
            i = i + 1

            edge_train.append([C1, C2, int(l[0])])
            train_src_l.append(C1)
            label_src.append(0)
            train_dst_l.append(C2)
            label_dst.append(0)
            train_ts_l.append(int(l[0]))

    return node_id, train_src_l, train_dst_l, train_ts_l, train_label_l, label_src, label_dst


node_id, te_src_l, te_dst_l, te_ts_l, te_label_l, label_src, label_dst = load_data(ALLEDGEPATH)
te_src_l = np.array(te_src_l)
te_dst_l = np.array(te_dst_l)
te_ts_l = np.array(te_ts_l)
te_label_l = np.array(te_label_l)
num_instance = len(te_src_l)
full_adj_list = adj_list.copy()
e_idx_l = list(np.arange(len(train_src_l)+len(te_src_l)))
for src, dst, eidx, ts in zip(te_src_l, te_dst_l, e_idx_l, te_ts_l):
    full_adj_list[src].append((dst, eidx, ts))
full_ngh_finder = temporalNeighbor(full_adj_list, uniform=UNIFORM)
lbg.ngh_finder = full_ngh_finder
idc = np.arange(num_instance)
num_batch = math.ceil(num_instance / BATCH_SIZE)

In [6]:
red_links = test('test for old nodes',lbg,R,num_instance,te_src_l,te_dst_l,te_ts_l,te_label_l,center)
rednodes_predict = set(red_links[0])|set(red_links[1])

2024-09-02 04:39:03,844 140564686231360 2644694426.py[line:2] - INFO: test for old nodes


100%|██████████| 1257/1257 [01:27<00:00, 14.36it/s]


2024-09-02 04:40:36,207 140564686231360 2644694426.py[line:35] - INFO: FPR_curve,: [0.00000000e+00 3.98069047e-07 3.86126975e-05 ... 9.99676768e-01
 9.99677564e-01 1.00000000e+00],TPR_curve,:[0. 0. 0. ... 1. 1. 1.]
2024-09-02 04:40:36,553 140564686231360 2644694426.py[line:37] - INFO: Epoch mean loss: 4.050813403884475
2024-09-02 04:40:36,896 140564686231360 2644694426.py[line:38] - INFO: Confusion Matrix:[[2435954   76173]
 [      4     494]]
2024-09-02 04:40:36,897 140564686231360 2644694426.py[line:41] - INFO: FPR: 0.03032211349187362
2024-09-02 04:40:36,898 140564686231360 2644694426.py[line:42] - INFO: TPR: 0.9919678714859438
0.03032211349187362 0.9919678714859438 0.9919213133941923
2024-09-02 04:40:36,898 140564686231360 2644694426.py[line:44] - INFO: test auc: 0.9919213133941923


ACG

In [7]:
data_name =  './data/LANL/TCG_LANL.txt'
precess_file_path =  './data/LANL/iniACG_LANL.txt'

In [8]:
filter_edge = set()
with open(ALLEDGEPATH, 'r', encoding='utf-8') as fi:
    while True:
        line = fi.readline()
        if not line:
            break
        l = line.strip().split(',')
        filter_edge.add((l[3], l[6]))

In [9]:
with open(precess_file_path, 'a') as sf:
    sf.truncate(0)
    sf.close()
with open(data_name, 'r', encoding='utf-8') as fi:
    fi.readline()
    sf = open(precess_file_path, 'a')
    while True:
        line = fi.readline()
        if not line:
            break
        l = line.strip().split(',')
        if ((l[3],l[6]) in filter_edge):
            continue
        else:
            sf.write(line)
sf.close()

In [17]:
nodes_count = dict(Counter(list(red_links[0])+list(red_links[1])))
#red_samenodes = [key for key,value in nodes_count.items()if value > 1]
node_sort=sorted(nodes_count.items(), key=lambda x: x[1], reverse=True)
red_samenodes = dict(node_sort[:80]).keys()
logger.info('num of predict red nodes: {}'.format(len(red_samenodes)))
nred_links = [(x,y) for x,y in zip(red_links[0],red_links[1])]
logger.info('num of predict red edges: {}'.format(len(nred_links)))
min_t = 0
max_t = 1296000
RR = 20
def inductive_link(red_samenodes,min_t,max_t):
    node_id = set()
    test_src_l = []
    test_dst_l = []
    test_ts_l = []
    test_label_l = []
    i = 0
    with open(precess_file_path, 'r', encoding='utf-8') as fi:
        fi.readline()
        while True:
            line = fi.readline()
            if not line:
                break
            l = line.strip().split(',')
            C1 = int(l[3].strip('C'))
            C2 = int(l[6].strip('C'))
            if (int(l[0]) < min_t or int(l[0]) > max_t):
                continue
            if (C1 in red_samenodes or C2 in red_samenodes):
                test_src_l.append(C1)
                test_dst_l.append(C2)
                test_ts_l.append(int(l[0]))
                node_id.add(C1)
                node_id.add(C2)
                str_line = str(l[0]) + ',' + str(l[1]) + ',' + str(l[2]) + ',' + str(l[3]) + ',' + str(l[6]) + '\n'
                if str_line in red_team_file:
                    test_label_l.append(1)
                else:
                    test_label_l.append(0)
            i = i + 1

    return node_id, test_src_l, test_dst_l, test_ts_l, test_label_l

test_node_id, test_src_l, test_dst_l, test_ts_l, test_label_l = inductive_link(red_samenodes,min_t,max_t)
logger.info('num of new nodes: {}'.format(len(set(test_node_id)-set(node_id))))
logger.info('num of new events: {}'.format(len(test_src_l)))
logger.info('num of red team: {}'.format(sum(test_label_l)))
test_src_l = np.array(test_src_l)
test_dst_l = np.array(test_dst_l)
test_ts_l = np.array(test_ts_l)
test_label_l = np.array(test_label_l)
num_instance = len(test_src_l)
full_adj_list = adj_list.copy()
e_idx_l = list(np.arange(len(train_src_l)+len(test_src_l)))
for src, dst, eidx, ts in zip(test_src_l, test_dst_l, e_idx_l, test_ts_l):
    full_adj_list[src].append((dst, eidx, ts))
full_ngh_finder = temporalNeighbor(full_adj_list, uniform=UNIFORM)
lbg.ngh_finder = full_ngh_finder
idc = np.arange(num_instance)
num_batch = math.ceil(num_instance / BATCH_SIZE)

2024-09-02 05:58:46,852 140564686231360 2600890899.py[line:5] - INFO: num of predict red nodes: 80
2024-09-02 05:58:46,866 140564686231360 2600890899.py[line:7] - INFO: num of predict red edges: 76667
2024-09-02 05:59:23,152 140564686231360 2600890899.py[line:45] - INFO: num of new nodes: 653
2024-09-02 05:59:23,155 140564686231360 2600890899.py[line:46] - INFO: num of new events: 2493378
2024-09-02 05:59:23,173 140564686231360 2600890899.py[line:47] - INFO: num of red team: 98


In [ ]:
new_center = update_center( test_src_l, test_dst_l,test_ts_l,lbg,num_instance,num_batch).to(device)
new_redlinks = test('test for new nodes', lbg, R, num_instance, test_src_l, test_dst_l, test_ts_l, test_label_l,new_center)

100%|██████████| 1247/1247 [01:35<00:00, 13.10it/s]

2024-09-02 06:01:13,762 140564686231360 2644694426.py[line:2] - INFO: test for new nodes



 51%|█████▏    | 642/1247 [00:53<00:49, 12.19it/s]